In [1]:
import os
import numpy as np
import cv2
import joblib
from skimage.feature import hog, local_binary_pattern
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [ ]:
def vread(basepath):
    spr_path = basepath + ".spr"
    sdt_path = basepath + ".sdt"

    with open(spr_path, "r") as f:
        ndim = int(f.readline().strip())
        if ndim != 2:
            raise ValueError("Only 2D SAR images supported")

        nc = int(f.readline().strip())
        _ = f.readline()
        _ = f.readline()
        nr = int(f.readline().strip())
        _ = f.readline()
        _ = f.readline()
        dtype_code = int(f.readline().strip())

    if dtype_code == 0:
        dtype = np.uint8
    elif dtype_code == 2:
        dtype = np.int32
    elif dtype_code == 3:
        dtype = np.float32
    elif dtype_code == 5:
        dtype = np.float64
    else:
        raise ValueError("Unknown dtype")

    with open(sdt_path, "rb") as f:
        data = np.frombuffer(f.read(), dtype=dtype)

    with open(sdt_path, "rb") as f:
        data = np.frombuffer(f.read(), dtype=dtype)

    # fix becuz github nub and changes the shape >:(
    expected_count = nr * nc
    if data.size > expected_count:
        data = data[:expected_count]

    img = data.reshape((nr, nc))
    return img

In [3]:
def load_lxyr(path):
    volcanoes = []
    with open(path, "r") as f:
        for line in f:
            parts = line.strip().split()
            cls = int(parts[0])
            x = float(parts[1])
            y = float(parts[2])
            r = float(parts[3])
            volcanoes.append((cls, x, y, r))
    return volcanoes

In [4]:
def extract_chips(img, volcanoes, size=64):
    h, w = img.shape
    half = size // 2
    
    pos_chips = []
    for _, x, y, r in volcanoes:
        cx, cy = int(x), int(y)

        x1 = max(0, cx - half)
        y1 = max(0, cy - half)
        x2 = min(w, cx + half)
        y2 = min(h, cy + half)

        chip = img[y1:y2, x1:x2]

        chip = cv2.copyMakeBorder(
            chip,
            top=max(0, size - chip.shape[0]),
            bottom=0,
            left=max(0, size - chip.shape[1]),
            right=0,
            borderType=cv2.BORDER_REFLECT
        )
        pos_chips.append(chip)

    return pos_chips

def extract_negative_chips(img, volcanoes, count=3, size=64):
    h, w = img.shape
    neg = []

    forbidden = [(int(x), int(y), r) for _, x, y, r in volcanoes]

    for _ in range(count * len(volcanoes)):
        while True:
            cx = np.random.randint(size, w - size)
            cy = np.random.randint(size, h - size)

            if all(np.hypot(cx - vx, cy - vy) > r + size/2 + 10 for vx, vy, r in forbidden):
                chip = img[cy-size//2:cy+size//2, cx-size//2:cx+size//2]
                neg.append(chip)
                break

    return neg

In [5]:
def preprocess(img):
    img = cv2.medianBlur(img, 3)

    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    img = clahe.apply(img.astype(np.uint8))

    return img

def extract_features(img):
    img = cv2.resize(img, (64, 64))

    fd, _ = hog(
        img,
        orientations=8,
        pixels_per_cell=(8,8),
        cells_per_block=(2,2),
        visualize=True,
        block_norm='L2'
    )

    lbp = local_binary_pattern(img, P=8, R=1, method='uniform')
    hist, _ = np.histogram(lbp, bins=10, range=(0,10), density=True)

    return np.hstack([fd, hist])

In [6]:
IMG_FOLDER = "./package/Images"
GT_FOLDER  = "./package/GroundTruths"

X = []
y = []

for fname in tqdm(os.listdir(GT_FOLDER)):
    if not fname.endswith(".lxyr"): 
        continue

    base = fname[:-5]  # remove .lxyr

    gt_path = os.path.join(GT_FOLDER, fname)
    volcanoes = load_lxyr(gt_path)

    img_path = os.path.join(IMG_FOLDER, base)
    img = vread(img_path)
    img = preprocess(img)

    # pos
    pos = extract_chips(img, volcanoes)
    for chip in pos:
        X.append(extract_features(chip))
        y.append(1)

    # neg
    neg = extract_negative_chips(img, volcanoes)
    for chip in neg:
        X.append(extract_features(chip))
        y.append(0)

100%|██████████| 278/278 [02:31<00:00,  1.83it/s]


In [7]:
X = np.array(X)
y = np.array(y)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

In [8]:
svm = SVC(kernel="rbf", probability=True, class_weight="balanced")
svm.fit(X_train, y_train)

print("Train accuracy:", svm.score(X_train, y_train))
print("Test accuracy:", svm.score(X_test, y_test))

joblib.dump(svm, "svm_model.pkl")
joblib.dump(scaler, "scaler.pkl")

print("Saved svm_model.pkl and scaler.pkl!")

Train accuracy: 0.9786184210526315
Test accuracy: 0.8379934210526315
Saved svm_model.pkl and scaler.pkl!
